# EDA - Cortex Real Estate Dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

# Add src to path if needed (standard when running in src/)
from services.portfolio.normalization import normalize_data, enrich_asset_metrics
from services.portfolio.asset_manager import AssetManagerAssistant

# Using glob to handle the complex filename
files = glob.glob("../data/*.parquet")
if files:
    df_raw = pd.read_parquet(files[0])
    df_norm = normalize_data(df_raw)
    df = enrich_asset_metrics(df_norm)
    assistant = AssetManagerAssistant(df)
    print("Data loaded, normalized, and enriched with asset metrics.")
else:
    print("No parquet file found.")

ImportError: cannot import name 'normalize_data' from 'services.portfolio.normalization' (/Users/ofircohen/Projects/CortexRE-assignment/src/services/portfolio/normalization.py)

## 1. Data Quality & Distribution

In [ ]:
df.info()
df.head()

In [ ]:
print("Missing values after normalization:")
df.isnull().sum()

## 2. Time-Series Analysis

We can now analyze trends at both the Monthly and Quarterly levels.

In [ ]:
# Monthly Profit Trend
plt.figure(figsize=(12, 6))
df_monthly = df.groupby('date')['profit'].sum().reset_index()
sns.lineplot(data=df_monthly, x='date', y='profit', marker='o')
plt.title('Monthly Total Profit Trend')
plt.grid(True)
plt.show()

In [ ]:
# Quarterly Profit Trend
plt.figure(figsize=(10, 5))
df_quarterly = df.groupby('quarter_start')['profit'].sum().reset_index()
sns.barplot(data=df_quarterly, x='quarter_start', y='profit', palette='viridis')
plt.title('Quarterly Total Profit')
plt.xticks(rotation=45)
plt.show()

## 3. Financial Breakdown

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='ledger_type', y='profit')
plt.title('Profit Distribution by Ledger Type')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
df_agg = df.groupby(['year', 'ledger_type'])['profit'].sum().reset_index()
sns.barplot(data=df_agg, x='year', y='profit', hue='ledger_type')
plt.title('Profit by Year and Ledger Type')
plt.show()

## 4. Property Comparison

In [ ]:
plt.figure(figsize=(12, 6))
noi_comparison = assistant.compare_properties('noi')
sns.barplot(x=noi_comparison.index, y=noi_comparison.values)
plt.title('NOI Comparison by Property')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
df_oer = df[df['property_name'] != 'Corporate/General']
sns.lineplot(data=df_oer, x='date', y='oer', hue='property_name')
plt.title('Operating Expense Ratio (OER) Over Time')
plt.show()

In [ ]:
property_names = df["property_name"].unique()
years = df["year"].unique()

for property_name in property_names:
    print(f"P&L for {property_name}:")
    for year in years:
        print(f"\t Year ({year})")
        property_pl = assistant.get_property_pl(property_name, int(year))
        if 'expenses' in property_pl:
            print(f"\t\t - Expenses {property_pl['expenses']}")
        if 'revenue' in property_pl:
            print(f"\t\t - Revenue {property_pl['revenue']}")
        if 'noi' in property_pl:
            print(f"\t\t - Net Operating Income {property_pl['noi']}")
    
    print(f"\nTop Expense Drivers for {property_name}:")
    top_expense_drivers = assistant.top_expense_drivers(property_name).head(5)
    print(top_expense_drivers)
    print("-" * 100)

## Advanced Portfolio Analysis

In [ ]:
print("--- Portfolio Financial Summary (2025) ---")
portfolio_2025 = assistant.get_portfolio_summary(2025)
print(f"Total Revenue:  {portfolio_2025['revenue']:,.2f}")
print(f"Total Expenses: {portfolio_2025['expenses']:,.2f}")
print(f"Portfolio NOI: {portfolio_2025['noi']:,.2f}")

### YoY Growth Analysis (NOI)

In [ ]:
# Calculate YoY Growth for NOI
noi_growth = assistant.get_growth_metrics('noi')

plt.figure(figsize=(12, 6))
sns.barplot(x=noi_growth.index, y=noi_growth.values * 100, palette='viridis')
plt.title("YoY NOI Growth % (2024 to 2025)")
plt.ylabel("Growth %")
plt.xlabel("Property")
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.show()

print("NOI Growth Metrics:")
print(noi_growth)

### Efficiency Metrics (OER)

In [ ]:
# Calculate OER for 2025 for all properties
oer_2025 = {}
for prop in df['property_name'].unique():
    if prop == 'Corporate/General':
        continue
    oer_2025[prop] = assistant.calculate_oer(prop, 2025)

oer_series = pd.Series(oer_2025).sort_values()

plt.figure(figsize=(12, 6))
sns.barplot(x=oer_series.index, y=oer_series.values, palette='magma')
plt.title("Operating Expense Ratio (OER) by Property - 2025")
plt.ylabel("OER (lower is more efficient)")
plt.xlabel("Property")
plt.show()

print("OER 2025 Metrics (Ratio):")
print(oer_series)

## Summary of Findings

### 1. Significant YoY NOI Decline
Across the entire portfolio, we observe a sharp decline in Net Operating Income (NOI) from 2024 to 2025 (averaging approx **-73%**). This suggests a potential systemic issue such as late revenue recognition in 2025 or a spike in capital expenditures at the end of 2024. Further investigation into specific ledger accounts for Q1 2025 is recommended.

### 2. Operational Efficiency (OER)
In 2025, most properties show an **Operating Expense Ratio (OER) of 0%**, indicating that significant operating costs haven't been recorded yet or the properties are running at an exceptionally high margin. **Building 120** is an outlier with an OER of **1.4%**, making it the only building with notable operating costs recorded in the 2025 dataset so far.

### 3. Portfolio Distribution
The portfolio NOI is concentrated in five primary buildings (**140, 180, 160, 120, 17**). 'Corporate/General' appears strictly as a cost center with significant negative profit (expenses) and no associated revenue, which aligns with standard institutional real estate accounting for overhead.

### 4. Data Quality Observations
- The dataset was successfully normalized from mixed-language descriptions.
- Date ranges are consistent across 2024 and 2025, but 2025 appears to have fewer expense entries compared to the same period in 2024, which might be dragging the OER down artificially.